In [1]:
using ImageIO, FileIO, ImageShow, Colors, Plots, StatsBase, FFTW, LinearAlgebra

In [2]:
raw = convert(Array{Float64}, convert(Array{Gray}, load("testing/raw.png")))
oneGS75 = convert(Array{Float64}, convert(Array{Gray}, load("testing/oneGS.png")))
oneGS70 = convert(Array{Float64}, convert(Array{Gray}, load("testing/oneGS70.png")))
oneGS77 = convert(Array{Float64}, convert(Array{Gray}, load("testing/oneGS77.png")))
twoGS7575 = convert(Array{Float64}, convert(Array{Gray}, load("testing/twoGS7575.png")))
twoGS7565 = convert(Array{Float64}, convert(Array{Gray}, load("testing/twoGS7565.png")))
twoGS7540 = convert(Array{Float64}, convert(Array{Gray}, load("testing/twoGS7540.png")))
twoGS7590 = convert(Array{Float64}, convert(Array{Gray}, load("testing/twoGS7590.png")))
twoGS9075 = convert(Array{Float64}, convert(Array{Gray}, load("testing/twoGS7590.png")))
twoGS7770 = convert(Array{Float64}, convert(Array{Gray}, load("testing/twoGS7770.png")));

In [3]:
function epq(img, p, q)
    irange = 1:8:4032-8
    jrange = 1:8:3024-8
    out = 0
    for i = irange
        for j = jrange
            out += abs(img[i+p,j+q]-img[i+p,j+q+1]-img[i+p+1,j+q]+img[i+p+1,j+q+1])
        end
    end
    return out
end

epq (generic function with 1 method)

In [34]:
function determine_origin(img)
    x = 0:7
    y = 0:7
    X = repeat(reshape(x, 1, :), length(y), 1)
    Y = repeat(y, 1, length(x))
    function inner(p,q)
        return(epq(img,p,q))
    end
    Z = map(inner, X, Y)
    return heatmap(0:7, 0:7, Z, c=cgrad([:blue, :cyan]), size = (500, 400), xlabel="p", ylabel="q")
end

determine_origin (generic function with 1 method)

In [35]:
determine_origin(raw);
png("Plots/raw_origin.png")

"Plots/raw_origin.png"

In [36]:
determine_origin(oneGS75);
png("Plots/oneGS75_origin.png")

"Plots/oneGS75_origin.png"

In [37]:
determine_origin(twoGS7575);
png("Plots/twoGS7575_origin.png")

"Plots/twoGS7575_origin.png"

In [38]:
determine_origin(twoGS7590);
png("Plots/twoGS7590_origin.png")

"Plots/twoGS7590_origin.png"

In [9]:
function get_Z1(img,p,q)
    out = []
    irange = p:8:4032-8
    jrange = q:8:3024-8
    for i = irange
        for j = jrange
            t = abs(img[i+3,j+3]-img[i+3,j+4]-img[i+4,j+3]+img[i+4,j+4])
            append!(out,t)
        end
    end
    return out
end
function get_Z2(img,p,q)
    out = []
    irange = p:8:4032-8
    jrange = q:8:3024-8
    for i = irange
        for j = jrange
            t = abs(img[i+7,j+7]-img[i+7,j+8]-img[i+8,j+7]+img[i+8,j+8])
            append!(out,t)
        end
    end
    return out
end
function zHistogram(img,p,q)
    h1 = get_Z1(img,p,q)
    h2 = get_Z2(img,p,q)
    histogram(h1, normalize=:probability, bins=100, ylabel="Probability", xlabel = "Neighbour Differences", label="Z'", size = (400, 300))
    histogram!(h2, normalize=:probability, bins=100, label="Z''")
end

zHistogram (generic function with 1 method)

In [10]:
function get_K(img,p,q)
    h1 = convert(Vector{Float64}, get_Z1(img,p,q))
    h2 = convert(Vector{Float64}, get_Z2(img,p,q))
    h1SB = normalize(fit(Histogram, h1, 0:0.005:0.25), mode=:probability)
    h2SB = normalize(fit(Histogram, h2, 0:0.005:0.25), mode=:probability)
    return plot(abs.(h2SB.weights-h1SB.weights), ylim=(-0.1, 1), xlim=(0,50), legend = false,size = (400, 300)),sum(abs.(h2SB.weights-h1SB.weights)),abs.(h2SB.weights-h1SB.weights)
end

get_K (generic function with 1 method)

In [11]:
zHistogram(raw,1,1)
png("Plots/raw_h12.png")

"Plots/raw_h12.png"

In [12]:
zHistogram(oneGS75,1,1)
png("Plots/oneGS75_h12")

"Plots/oneGS75_h12.png"

In [13]:
rawAbsDiff,k = get_K(raw,1,1)
println(k)
rawAbsDiff[1];
png("Plots/raw_h12_diff.png")

0.009555399697306882


"Plots/raw_h12_diff.png"

In [14]:
oneAbsDiff,k = get_K(oneGS75,1,1)
println(k)
oneAbsDiff[1];
png("Plots/oneGS75_h12_diff.png")

1.2163412100342244


"Plots/oneGS75_h12_diff.png"

In [15]:
function dct_coeffs(img,m,n)
    irange = 1:8:4032-8
    jrange = 1:8:3024-8
    out = []
    for i = irange
        for j = jrange
            x = img[i:i+7,j:j+7]
            x .*= 255
            x .-= 128
            append!(out,dct(x)[m,n])
        end
    end
    return out
end
function coeff_hist(img, isGray, colour, m, n)
    if isGray
        coeffs = dct_coeffs(img,m,n)
    else
        coeffs = dct_coeffs(colour.(img),m,n)
    end
    coeffs = [x for x in coeffs if abs(x) < 30]
    graph = stephist(coeffs, bins=300, xlabel="Coefficient values", label="m=$m, n=$n",size = (400, 300))
    
    return graph
end

coeff_hist (generic function with 1 method)

In [16]:
coeff_hist(raw,true,0,1,1);
png("Plots/raw_coeff_11.png")

"Plots/raw_coeff_11.png"

In [17]:
coeff_hist(raw,true,0,1,2);
png("Plots/raw_coeff_12.png")

"Plots/raw_coeff_12.png"

In [18]:
coeff_hist(oneGS75,true,0,1,1);
png("Plots/oneGS75_coeff_11.png")

"Plots/oneGS75_coeff_11.png"

In [19]:
coeff_hist(oneGS75,true,0,1,2);
png("Plots/oneGS75_coeff_12.png")

"Plots/oneGS75_coeff_12.png"

In [20]:
coeff_hist(twoGS7770,true,0,1,2);
png("Plots/twoGS7770_coeff_12.png")

"Plots/twoGS7770_coeff_12.png"

In [21]:
coeff_hist(oneGS77,true,0,1,2);
png("Plots/oneGS77_coeff_12.png")

"Plots/oneGS77_coeff_12.png"

In [22]:
coeff_hist(oneGS70,true,0,1,2);
png("Plots/oneGS70_coeff_12.png")

"Plots/oneGS70_coeff_12.png"

In [23]:
test1 = transpose(convert(Array{Float64}, convert(Array{Gray}, load("test1.png"))))
test2 = convert(Array{Float64}, convert(Array{Gray}, load("test2.png")))
test3 = convert(Array{Float64}, convert(Array{Gray}, load("test3.png")))
test1c = transpose(convert(Array{Float64}, convert(Array{Gray}, load("test1c.png"))))
test2c = convert(Array{Float64}, convert(Array{Gray}, load("test2c.png")))
test3c = convert(Array{Float64}, convert(Array{Gray}, load("test3c.png")));

In [39]:
determine_origin(test1)
png("Plots/test1_origin.png")
determine_origin(test2)
png("Plots/test2_origin.png")
determine_origin(test3)
png("Plots/test3_origin.png")
determine_origin(test1c)
png("Plots/test1c_origin.png")
determine_origin(test2c)
png("Plots/test2c_origin.png")
determine_origin(test3c);
png("Plots/test3c_origin.png")

"Plots/test3c_origin.png"

In [25]:
test1AbsDiff = get_K(test1,1,1)
println(test1AbsDiff[2])
test1AbsDiff[1];
png("Plots/test1_h12_diff.png")

test2AbsDiff = get_K(test2,1,1)
println(test2AbsDiff[2])
test2AbsDiff[1];
png("Plots/test2_h12_diff.png")

test3AbsDiff = get_K(test3,1,1)
println(test3AbsDiff[2])
test3AbsDiff[1];
png("Plots/test3_h12_diff.png")

test1cAbsDiff = get_K(test1c,1,1)
println(test1cAbsDiff[2])
test1cAbsDiff[1];
png("Plots/test1c_h12_diff.png")

test2cAbsDiff = get_K(test2c,1,1)
println(test2cAbsDiff[2])
test2cAbsDiff[1];
png("Plots/test2c_h12_diff.png")

test3cAbsDiff = get_K(test3c,1,1)
println(test3cAbsDiff[2])
test3cAbsDiff[1];
png("Plots/test3c_h12_diff.png")

plot(test1AbsDiff[3], ylim=(-0.1, 1), xlim=(0,50), label = "test1")
plot!(test2AbsDiff[3], ylim=(-0.1, 1), xlim=(0,50), label = "test2")
plot!(test3AbsDiff[3], ylim=(-0.1, 1), xlim=(0,50), label = "test3")
plot!(test1cAbsDiff[3], ylim=(-0.1, 1), xlim=(0,50), label = "test1c")
plot!(test2cAbsDiff[3], ylim=(-0.1, 1), xlim=(0,50), label = "test2c")
plot!(test3cAbsDiff[3], ylim=(-0.1, 1), xlim=(0,50), label = "test3c");
png("Plots/test_h12_diffs")

0.007674370320079288
1.184289488532993
1.173067694627988
1.1208683368149075
1.2166998011928432
0.00984016326444519


"Plots/test_h12_diffs.png"

In [26]:
Ks = [test1AbsDiff[2], test2AbsDiff[2], test3AbsDiff[2], test1cAbsDiff[2], test2cAbsDiff[2], test3cAbsDiff[2]];
Xs = ["test1", "test2", "test3", "test1c", "test2c", "test3c"]
bar(Xs,Ks, label="K")
plot!(Xs,[0.25 for i in Xs], label="Threshold", size = (400, 300));
png("Plots/test_Ks.png")

"Plots/test_Ks.png"

In [27]:
coeff_hist(test1,true,0,1,2)
png("Plots/test1_coeff_12.png")
coeff_hist(test2,true,0,1,2)
png("Plots/test2_coeff_12.png")
coeff_hist(test3,true,0,1,2)
png("Plots/test3_coeff_12.png")
coeff_hist(test1c,true,0,1,2)
png("Plots/test1c_coeff_12.png")
coeff_hist(test2c,true,0,1,2)
png("Plots/test2c_coeff_12.png")
coeff_hist(test3c,true,0,1,2);
png("Plots/test3c_coeff_12.png")

"Plots/test3c_coeff_12.png"